In [ ]:
%pylab inline

In [ ]:
from PIL import Image

In [ ]:
stepdata=1 # sparsify data before computing
k=21 # choose number of clusters
maxiterations=1000 # number of iterations
whichalgo=1 # which algorithm
whichimage=0 # which image to start with
doubleprecision=0 # if double precision or not

In [ ]:
# also control "step" that sets how much of final plot to show (too many points takes too long even if kmeans if fast)
if whichimage==0:
    logo = Image.open("h2o-logo.jpg");step=13
if whichimage==1: 
    logo = Image.open("low_h2o-logo.jpg");step=2
if whichimage==2:
    logo = Image.open("lowlow_h2o-logo.jpg");step=1

In [ ]:
data = array(logo)

In [ ]:
imshow(data)

In [ ]:
data.shape

In [ ]:
N = int(data.shape[0]*data.shape[1]/2)
seed(777)
space = concatenate((randint(0, data.shape[0], N)[:, newaxis], randint(0, data.shape[1], N)[:, newaxis]), axis=1)

In [ ]:
uspace = zeros_like(data)
uspace[space[:, 0], space[:, 1]] = 1

In [ ]:
imshow(uspace)

In [ ]:
dots = data * uspace

In [ ]:
imshow(dots)

In [ ]:
dataset = vstack(nonzero(dots)).T

In [ ]:
#numpy.set_printoptions(threshold=numpy.nan)
#print(dataset)

In [ ]:
dataset = dataset[::stepdata]    
np.isfortran(dataset)
print(type (dataset))
print(dataset.shape)

rows=np.shape(dataset)[0]
print("rows=%d" % (rows))
labels = np.random.randint(rows, size=rows) % k

if doubleprecision==1:
    dataset = dataset.astype(float64)
    labels = labels.astype(float32)
else:
    dataset = dataset.astype(float32)
    labels = labels.astype(float32)

for i in range(k):
    print(i in labels)

In [ ]:
print(labels)
print(dataset)

In [ ]:
if whichalgo==0:
    import libKMCUDA
    %time cents, asses = libKMCUDA.kmeans_cuda(dataset, k, seed=77, device=0, yinyang_t=0,tolerance=1e-3,verbosity=1)
    # 3m25s on 1 titanX pascal with stepdata=1 , k=21 , maxiterations=1000 , whichalgo=0 , whichimage=0
if whichalgo==1:
    import h2ogpuml
    rows=np.shape(dataset)[0]
    np.random.seed(1234)
    n_gpus=1
    init_from_labels=0 # so auto-generate initial centroids, no need to initialize labels or centroids
    model = h2ogpuml.KMeans(n_gpus=n_gpus, n_clusters=k, tol=1e-7, max_iter=maxiterations,init_from_labels=init_from_labels)
    %time model.fit(dataset, labels)
    cents = model.cluster_centers_
    asses = model.predict(dataset)
    # 14s on 1 titanX pascal with stepdata=1 , k=21 , maxiterations=1000 , whichalgo=1 , whichimage=0
    # 26s on 1 titanX maxwell with stepdata=1 , k=21 , maxiterations=1000 , whichalgo=1 , whichimage=0
    # 4s on 4 titanX pascals with stepdata=1 , k=21 , maxiterations=1000 , whichalgo=1 , whichimage=0

if whichalgo==2:
    #hist(dataset)
    np.savetxt("data.csv",dataset,delimiter=",")
    # see /opt/intel/compilers_and_libraries_2017/linux/daal/examples/python/source/kmeans
    # source /opt/intel/compilers_and_libraries_2017/linux/daal/bin/daalvars.sh intel64
    # cp -a /opt/intel/compilers_and_libraries_2017/linux/daal/examples/python/source/utils  ~/.pyenv/versions/3.6.1/lib/python3.6/site-packages/
    # sudo chown -R $USER:$USER /opt/intel
    # cd /opt/intel/compilers_and_libraries_2017/linux/daal/pydaal_sources ; python setup.py install
    
    # 59s with stepdata=1 , k=21 , maxiterations=1000 , whichalgo=2 , whichimage=0 physics.umd.edu
    # 50s with stepdata=1 , k=21 , maxiterations=1000 , whichalgo=2 , whichimage=0 pseudotensor
    from daal.data_management import (
        FileDataSource, DataSourceIface
    )
    from daal.algorithms.kmeans import (
        Batch_Float32LloydDense, init, data, inputCentroids, assignments, centroids, goalFunction
    )
    from os.path import join as jp
    datasetFileName = jp('data.csv')
    
    import inspect, sys, os.path
    utils_folder = os.path.realpath(os.path.abspath(jp(os.path.split(inspect.getfile(inspect.currentframe()))[0], "..")))
    if utils_folder not in sys.path:
        sys.path.insert(0, utils_folder)
    from utils import printNumericTable
    from utils import isFull
        
    from daal.data_management import (
        CSRNumericTable, NumericTableIface, readOnly, BlockDescriptor,
        packed_mask, HomogenNumericTable, KeyValueDataCollection,
        DataSourceIface, FileDataSource, HomogenTensor, SubtensorDescriptor,
    )

    # K-Means algorithm parameters
    nClusters = k
    nIterations = maxiterations

    # Initialize FileDataSource to retrieve the input data from a .csv file
    dataSource = FileDataSource(
        datasetFileName,
        DataSourceIface.doAllocateNumericTable,
        DataSourceIface.doDictionaryFromContext
    )

    # Retrieve the data from the input file
    dataSource.loadDataBlock()

    # Get initial clusters for the K-Means algorithm (can change Float32 to Float64)
    initAlg = init.Batch_Float32RandomDense(nClusters)

    initAlg.input.set(init.data, dataSource.getNumericTable())

    res = initAlg.compute()
    %time centroidsResult = res.get(init.centroids)

    # Create an algorithm object for the K-Means algorithm (can change Float32 to Float64)
    algorithm = Batch_Float32LloydDense(nClusters, nIterations)

    algorithm.input.set(data, dataSource.getNumericTable())
    algorithm.input.set(inputCentroids, centroidsResult)

    %time res = algorithm.compute()

    # Print the clusterization results
    asses0 = res.get(assignments)
    cents0 = res.get(centroids)

    num_rows = asses0.getNumberOfRows()
    num_cols = asses0.getNumberOfColumns()
    layout = asses0.getDataLayout()
    if isFull(layout) or layout == NumericTableIface.csrArray:
        datatype=0
    else:
        datatype=1
    print("asses: num_rows=%d num_cols=%d datatype=%d" % (num_rows,num_cols,datatype))
    block = BlockDescriptor()
    asses0.getBlockOfRows(0, num_rows, readOnly, block)
    asses=block.getArray()
    
    num_rows = cents0.getNumberOfRows()
    num_cols = cents0.getNumberOfColumns()
    layout = cents0.getDataLayout()
    if isFull(layout) or layout == NumericTableIface.csrArray:
        datatype=0
    else:
        datatype=1
    print("cents: num_rows=%d num_cols=%d datatype=%d" % (num_rows,num_cols,datatype))
    block2 = BlockDescriptor()
    cents0.getBlockOfRows(0, num_rows, readOnly, block2)
    cents=block2.getArray()
    
    printNumericTable(res.get(assignments), "First k cluster assignments:", k)
    printNumericTable(res.get(centroids), "First 3 dimensions of centroids:", k, 3)
    printNumericTable(res.get(goalFunction), "Goal function value:")

In [ ]:
print(type (dataset))

In [ ]:
print(cents)

In [ ]:
print(asses)

In [ ]:
np.ndarray.flatten(asses)

In [ ]:
rcParams["figure.figsize"] = (10, 10)
scatter(dataset[::step, 1], dots.shape[0] - dataset[::step, 0], c=asses[::step], edgecolors="none")
scatter(cents[:, 1], dots.shape[0] - cents[:, 0], c="black", s=40, edgecolors="none")
xlim((0, dots.shape[1]))
ylim((0, dots.shape[0]))
axis("off")
savefig("sourced.png", transparent=True, bbox_inches=0, pad_inches=0)